In [7]:
import pickle
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_json('generated_data.json')
data

,proj_id,team_size,budget,workload,completion_time,assigned_manager,tasks
0,1,2,10125.05,medium,155,Victoria Dunn,"[{'id': 1, 'description': 'Document integratio..."
1,2,7,10158.91,low,141,Victoria Dunn,"[{'id': 1, 'description': 'Implement integrati..."
2,3,8,12946.33,low,111,Joseph Parker,"[{'id': 1, 'description': 'Implement backend l..."
3,4,2,14865.58,medium,70,George Shannon,"[{'id': 1, 'description': 'Implement user inte..."
4,5,6,11776.35,high,166,Joseph Parker,"[{'id': 1, 'description': 'Document backend lo..."
...,...,...,...,...,...,...,...
995,996,7,14304.52,medium,150,Adam Armstrong,"[{'id': 1, 'description': 'Test user interface..."
996,997,6,11639.45,low,74,Adam Armstrong,"[{'id': 1, 'description': 'Test integration mo..."
997,998,3,11636.96,medium,263,David Johnson,"[{'id': 1, 'description': 'Optimize integratio..."
998,999,7,13117.18,high,177,Henry Carr,"[{'id': 1, 'description': 'Design database sch..."


In [3]:
data.isnull().sum(axis = 0)

proj_id                      1502
team_size                    1502
budget                       1502
workload                     1502
completion_time              1502
assigned_manager             1502
tasks__id                       0
tasks__description              0
tasks__status                   0
tasks__person_assigned          0
tasks__due_date                 0
tasks__estimated_duration       0
dtype: int64

In [28]:
import pandas as pd
import json

# Load JSON data
with open('generated_data.json') as f:
    data = json.load(f)
    
# Normalize JSON data to flatten nested structure
df = pd.json_normalize(data, 'tasks', ['proj_id', 'team_size', 'budget', 'workload', 'completion_time', 'assigned_manager'])

# Group by project ID and count the number of tasks
task_counts = df.groupby('proj_id').size().reset_index(name='task_count')

# Drop the 'tasks' column from the DataFrame
df.drop(columns=['id', 'description', 'status', 'person_assigned', 'due_date', 'estimated_duration'], inplace=True)

# Merge task_counts into the DataFrame on 'proj_id'
df = df.merge(task_counts, on='proj_id')

# Delete the duplicates
df = df.drop_duplicates(subset='assigned_manager')

# Display the updated DataFrame
display(df)


,proj_id,team_size,budget,workload,completion_time,assigned_manager,task_count
0,1,2,10125.05,medium,155,Victoria Dunn,1
2,3,8,12946.33,low,111,Joseph Parker,3
5,4,2,14865.58,medium,70,George Shannon,3
12,7,8,10714.36,low,156,Margaret Clayton,3
15,8,7,12982.05,medium,225,Nicole Lewis,3
18,9,2,14352.59,medium,357,Daniel Romero,2
20,10,5,11197.26,low,320,Ashley Lyons,4
24,11,8,10605.02,low,62,Daniel Lopez,2
26,12,9,11190.02,high,278,Joseph Johnson,2
28,13,7,11771.49,medium,269,Taylor Delgado,1


In [29]:
import pyarrow.parquet as pq
import pyarrow as pa
df.to_parquet("cleaned_generated_data.parquet")

In [30]:
cl_data = pd.read_parquet("cleaned_generated_data.parquet")

In [31]:
#df1 = pd.get_dummies(cl_data, columns=['homeworld', 'unit_type'])
#display(df1)
df1 = cl_data
X = df1[['task_count','team_size']]
y = df1['completion_time']

X_train, X_test, y_train, y_test,= train_test_split(X, y, test_size=0.2, random_state = 42)

model = LogisticRegression()
model = model.fit(X_train,y_train)

pred = model.predict(X_test)
pred

accuracy = model.score(X, y)
accuracy

C:\Users\wasadmin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.22